Auteur: Michaël Leroy

 Interpolation et ajout TMJA à toutes les routes

 [+] Méthodologie:

        par itérations successives:

            - deux dictionnaires regroupant les edges connus et ceux à interpoler

            - pour tous les inconnus, on prends la liste des edges adjacents, calcul d'une valeur aproximée en fonction du nombre de routes qui se branchent à chaque extrémité, multiplication par un coeff d'afaiblissement lié au nobre d'itération nécessaires pour atteindre le tronçon

 [-] Limitation:

        - calcul de la valeur déterminé un peu au pif, à part pour l'affaiblissement( reprise exponetial decay comme pour un learning rate), du coup pourquoi pas essayer avec un cosine decay et tant qu'a faire on pourrait stepper warmup, plateau, period et min. 
        - difficile de vérifier autrement que par une connaissance 'métier' de la circulation d'une région que l'on connait...     


Ajout d'une 'sauvegarde' entre chaque itération pour reprendre au step pécédent en cas de vscode qui tient absoluement à fermer la fenêtre qui ne répond pas...            

In [ ]:
class CFG:
    area = [
        'Auvergne-Rhône-Alpes',
    'Bourgogne-Franche-Comté',
    'Brittany',
    'Centre-Val de Loire',
    'Grand Est',
    'Hauts-de-France',
    'Île-de-France',
    'Normandie',
    'Nouvelle-Aquitaine',
    'Occitanie',
    'Pays de la Loire',
    'Provence-Alpes-Côte d\'Azur'
]
    area_name = 'france'      #'_'.join(area)

    

    bornes = False

    viz = False

    roads = (
        '|motorway'
        '|motorway_link'
        '|trunk'
        '|trunk_link'
        '|primary'
        '|primary_link'
        '|secondary'
        '|secondary_link'
        '|tertiary'
         '|tertiary_link'
    )

        # '|primary'
        # '|motorway_link'
        
        # '|primary_link'

        # '|secondary'
        # '|secondary_link'
        
        
        #  '|tertiary'
        #  '|tertiary_link'

    data_path = './osm_datas/'

import joblib
joblib.dump(CFG.area, 'CFG.pkl')    
import os
os.environ['USE_PYGEOS'] = '0'
%load_ext autoreload
%autoreload



%aimport OSM_helper_functions
%aimport OCM_helper_functions

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import osmnx as ox
import networkx as nx
from tqdm.notebook import tqdm
from OSM_helper_functions import *
from OCM_helper_functions import *


ox.settings.log_console=True
ox.settings.use_cache=True
''' To retrieve http data from OSMnx and cache them locally
    before building graph. Raise and exception if requests
    are all done. intercept then set to False and retry.'''
# ox.settings.cache_only_mode=True 

Administratives = {
    'Auvergne-Rhône-Alpes': ['Ain', 'Allier', 'Ardèche', 'Cantal', 'Drôme', 'Isère', 'Loire', 'Haute-Loire', 'Puy-de-Dôme', 'Rhône', 'Savoie', 'Haute-Savoie'],
    'Bourgogne-Franche-Comté': ['Côte-d\'Or', 'Doubs', 'Jura', 'Nièvre', 'Haute-Saône', 'Saône-et-Loire', 'Yonne', 'Territoire de Belfort'],
    'Brittany': ['Côtes-d\'Armor', 'Finistère', 'Ille-et-Vilaine', 'Morbihan'],
    'Centre-Val de Loire': ['Cher', 'Eure-et-Loir', 'Indre', 'Indre-et-Loire', 'Loir-et-Cher', 'Loiret'],
    # 'Corse': ['Corse-du-Sud', 'Haute-Corse'],
    'Grand Est': ['Ardennes', 'Aube', 'Marne', 'Haute-Marne', 'Meurthe-et-Moselle', 'Meuse', 'Moselle', 'Bas-Rhin', 'Haut-Rhin', 'Vosges'],
    'Hauts-de-France': ['Aisne', 'Nord', 'Oise', 'Pas-de-Calais', 'Somme'],
    'Île-de-France': ['Paris', 'Seine-et-Marne', 'Yvelines', 'Essonne', 'Hauts-de-Seine', 'Seine-Saint-Denis', 'Val-de-Marne', 'Val-d\'Oise'],
    'Normandie': ['Calvados', 'Eure', 'Manche', 'Orne', 'Seine-Maritime'],
    'Nouvelle-Aquitaine': ['Charente', 'Charente-Maritime', 'Corrèze', 'Creuse', 'Dordogne', 'Gironde', 'Landes', 'Lot-et-Garonne', 'Pyrénées-Atlantiques', 'Deux-Sèvres', 'Vienne', 'Haute-Vienne'],
    'Occitanie': ['Ariège', 'Aude', 'Aveyron', 'Gard', 'Haute-Garonne', 'Gers', 'Hérault', 'Lot', 'Lozère', 'Hautes-Pyrénées', 'Pyrénées-Orientales', 'Tarn', 'Tarn-et-Garonne'],
    'Pays de la Loire': ['Loire-Atlantique', 'Maine-et-Loire', 'Mayenne', 'Sarthe', 'Vendée'],
    'Provence-Alpes-Côte d\'Azur': ['Alpes-de-Haute-Provence', 'Hautes-Alpes', 'Alpes-Maritimes', 'Bouches-du-Rhône', 'Var', 'Vaucluse']
}

Regions = [
    'Auvergne-Rhône-Alpes',
    'Bourgogne-Franche-Comté',
    'Brittany',
    'Centre-Val de Loire',
    # 'Corse',
    'Grand Est',
    'Hauts-de-France',
    'Île-de-France',
    'Normandie',
    'Nouvelle-Aquitaine',
    'Occitanie',
    'Pays de la Loire',
    'Provence-Alpes-Côte d\'Azur'
]

Name2num = {
    'Ain': '01',
    'Aisne': '02',
    'Allier': '03',
    'Alpes-de-Haute-Provence': '04',
    'Hautes-Alpes': '05',
    'Alpes-Maritimes': '06',
    'Ardèche': '07',
    'Ardennes': '08',
    'Ariège': '09',
    'Aube': '10',
    'Aude': '11',
    'Aveyron': '12',
    'Bouches-du-Rhône': '13',
    'Calvados': '14',
    'Cantal': '15',
    'Charente': '16',
    'Charente-Maritime': '17',
    'Cher': '18',
    'Corrèze': '19',
    'Corse-du-Sud': '2A',
    'Haute-Corse': '2B',
    'Côte-d\'Or': '21',
    'Côtes-d\'Armor': '22',
    'Creuse': '23',
    'Dordogne': '24',
    'Doubs': '25',
    'Drôme': '26',
    'Eure': '27',
    'Eure-et-Loir': '28',
    'Finistère': '29',
    'Gard': '30',
    'Haute-Garonne': '31',
    'Gers': '32',
    'Gironde': '33',
    'Hérault': '34',
    'Ille-et-Vilaine': '35',
    'Indre': '36',
    'Indre-et-Loire': '37',
    'Isère': '38',
    'Jura': '39',
    'Landes': '40',
    'Loir-et-Cher': '41',
    'Loire': '42',
    'Haute-Loire': '43',
    'Loire-Atlantique': '44',
    'Loiret': '45',
    'Lot': '46',
    'Lot-et-Garonne': '47',
    'Lozère': '48',
    'Maine-et-Loire': '49',
    'Manche': '50',
    'Marne': '51',
    'Haute-Marne': '52',
    'Mayenne': '53',
    'Meurthe-et-Moselle': '54',
    'Meuse': '55',
    'Morbihan': '56',
    'Moselle': '57',
    'Nièvre': '58',
    'Nord': '59',
    'Oise': '60',
    'Orne': '61',
    'Pas-de-Calais': '62',
    'Puy-de-Dôme': '63',
    'Pyrénées-Atlantiques': '64',
    'Hautes-Pyrénées': '65',
    'Pyrénées-Orientales': '66',
    'Bas-Rhin': '67',
    'Haut-Rhin': '68',
    'Rhône': '69',
    'Haute-Saône': '70',
    'Saône-et-Loire': '71',
    'Sarthe': '72',
    'Savoie': '73',
    'Haute-Savoie': '74',
    'Paris': '75',
    'Seine-Maritime': '76',
    'Seine-et-Marne': '77',
    'Yvelines': '78',
    'Deux-Sèvres': '79',
    'Somme': '80',
    'Tarn': '81',
    'Tarn-et-Garonne': '82',
    'Var': '83',
    'Vaucluse': '84',
    'Vendée': '85',
    'Vienne': '86',
    'Haute-Vienne': '87',
    'Vosges': '88',
    'Yonne': '89',
    'Territoire de Belfort': '90',
    'Essonne': '91',
    'Hauts-de-Seine': '92',
    'Seine-Saint-Denis': '93',
    'Val-de-Marne': '94',
    'Val-d\'Oise': '95'
}


AREA = []
for area in CFG.area:
    AREA.extend(Administratives[area] )
AREA

In [ ]:
import joblib
import os

# Map str to integers and save the graph to disk
def map_save_graph(graph, filename, datapath='./datas', dataframes=True):
    # mapping dicts
    keys = []; values = []
    for n, node in enumerate(graph.nodes()):
        keys.append(n); values.append(node)
    int2id = dict(zip(keys, values))
    id2int = dict(zip(values, keys))    
    # Save the dicts to disk
    joblib.dump((int2id, id2int), os.path.join(datapath, f'{filename}_mapping_dicts.pkl'))
    graph = nx.relabel_nodes(graph, id2int)
    # Save the graph to disk
    ox.save_graphml(graph, os.path.join(datapath, filename))
    if dataframes:
        joblib.dump( ox.graph_to_gdfs(graph), os.path.join(datapath, f'{filename}_gdfs.pkl'))



# Load the mapping and graph from disk
def map_load_graph(filename, datapath='./datas'):
    try:
        int2id, id2int = joblib.load(os.path.join(datapath, f'{filename}_mapping_dicts.pkl'))
        return nx.relabel_nodes(ox.load_graphml(os.path.join(datapath, filename)), int2id), id2int, int2id
    except ValueError:
        print(ValueError)

# Load the mapping and graph from disk
def map_load_graph_gdfs(filename, datapath='./datas'):
    try:
        return joblib.load(os.path.join(datapath, f'{filename}_gdfs.pkl'))
    except ValueError:
        print(ValueError)        

    
# from matplotlib.pyplot import figure, show
import numpy

class ZoomPan:
    def __init__(self):
        self.press = None
        self.cur_xlim = None
        self.cur_ylim = None
        self.x0 = None
        self.y0 = None
        self.x1 = None
        self.y1 = None
        self.xpress = None
        self.ypress = None


    def zoom_factory(self, ax, base_scale = 2.):
        def zoom(event):
            cur_xlim = ax.get_xlim()
            cur_ylim = ax.get_ylim()

            xdata = event.xdata # get event x location
            ydata = event.ydata # get event y location

            if event.button == 'down':
                # deal with zoom in
                scale_factor = 1 / base_scale
            elif event.button == 'up':
                # deal with zoom out
                scale_factor = base_scale
            else:
                # deal with something that should never happen
                scale_factor = 1
                print (event.button)

            new_width = (cur_xlim[1] - cur_xlim[0]) * scale_factor
            new_height = (cur_ylim[1] - cur_ylim[0]) * scale_factor

            relx = (cur_xlim[1] - xdata)/(cur_xlim[1] - cur_xlim[0])
            rely = (cur_ylim[1] - ydata)/(cur_ylim[1] - cur_ylim[0])

            ax.set_xlim([xdata - new_width * (1-relx), xdata + new_width * (relx)])
            ax.set_ylim([ydata - new_height * (1-rely), ydata + new_height * (rely)])
            ax.figure.canvas.draw()

        fig = ax.get_figure() # get the figure of interest
        fig.canvas.mpl_connect('scroll_event', zoom)

        return zoom

    def pan_factory(self, ax):
        def onPress(event):
            if event.inaxes != ax: return
            self.cur_xlim = ax.get_xlim()
            self.cur_ylim = ax.get_ylim()
            self.press = self.x0, self.y0, event.xdata, event.ydata
            self.x0, self.y0, self.xpress, self.ypress = self.press

        def onRelease(event):
            self.press = None
            ax.figure.canvas.draw()

        def onMotion(event):
            if self.press is None: return
            if event.inaxes != ax: return
            dx = event.xdata - self.xpress
            dy = event.ydata - self.ypress
            self.cur_xlim -= dx
            self.cur_ylim -= dy
            ax.set_xlim(self.cur_xlim)
            ax.set_ylim(self.cur_ylim)

            ax.figure.canvas.draw()

        fig = ax.get_figure() # get the figure of interest

        # attach the call back
        fig.canvas.mpl_connect('button_press_event',onPress)
        fig.canvas.mpl_connect('button_release_event',onRelease)
        fig.canvas.mpl_connect('motion_notify_event',onMotion)

        #return the function
        return onMotion



import pandas as pd
from matplotlib import pyplot as plt

def graphplot(graph, edg_attr, nd_attr=None):
    # graph todataframe
    edges = ox.graph_to_gdfs(graph, nodes=False)
    try:
        edges[edg_attr].fillna(0, inplace=True)
    except:
        return print(f"Graph as no attribute '{edg_attr}'")
    # count attributes values
    edge_types = edges[edg_attr].value_counts()
    print('Color number:', len(edge_types))
    # get a colormap
    color_list = ox.plot.get_colors(n=len(edge_types), cmap='plasma_r',  alpha=0.7)
    # make a color mapper dict
    color_mapper = dict(
        zip(
        [l[0]  if type(l) == list else l for l in edge_types.index], 
        color_list
        )
    )
    # set the color for attribute less edges
    color_mapper[0.0] = (.0,.0,.0,0.1)
    # get the color for each edge based on its attribute
    attr = [ d.get(edg_attr, 0) for u, v, k, d in graph.edges(keys=True, data=True)]
    attr = [ l[0] if type(l) == list else l for l in attr]
    ec = [color_mapper[l] for l in attr]
    # set nodes color
    nc = [(.0,.0,1.0,0.9) for _ in graph.nodes()] if nd_attr==None else [d.get(nd_attr,(.0,.0,1.0,.9)) for _, d in graph.nodes(data=True)]
    # plot graph
    fig, ax = plt.subplots()
    fig.suptitle(f'{selected_regions} {edg_attr}')
    ox.plot_graph(graph, 
                  edge_color=ec, 
                  node_color=nc,
                  node_size=2 if nd_attr==None else 10, 
                  ax=ax,
                  show=False)
    scale = 1.1
    zp = ZoomPan()
    figZoom = zp.zoom_factory(ax, base_scale = scale)
    figPan = zp.pan_factory(ax)
    plt.show()


# fig = figure()

# ax = fig.add_subplot(111, xlim=(0,1), ylim=(0,1), autoscale_on=False)

# ax.set_title('Click to zoom')
# x,y,s,c = numpy.random.rand(4,200)
# s *= 200

# ax.scatter(x,y,s,c)
# scale = 1.1
# zp = ZoomPan()
# figZoom = zp.zoom_factory(ax, base_scale = scale)
# figPan = zp.pan_factory(ax)
# show()

import folium
from folium.plugins import MarkerCluster


def graph_correction(graph, viz=True):
    alone = {}
    single = {}
    # dual = {}

    node_dict = graph.nodes('infos')
    for node in graph.nodes():
        # edges = G1.edges(node)
        # print(len(edges))
        if len(graph.edges(node))==0:
            alone[node] = node_dict[node]
        elif len(graph.edges(node))==1:
            single[node] = node_dict[node]    
        # break
    
    if viz:
        m = ox.folium.plot_graph_folium(graph, tiles='OpenStreetMap')
        Agroup = MarkerCluster(name="Alone nodes")
        Sgroup = MarkerCluster(name="Single nodes")

        for  n in alone.keys():
            x = nx.get_node_attributes(graph,'x')[n]
            y = nx.get_node_attributes(graph,'y')[n]
            Agroup.add_child(
                folium.Marker(
                    location=[y, x],
                    popup=n, icon=folium.Icon(color='red')
                    )
                )
        for  n in single.keys():
            x = nx.get_node_attributes(graph,'x')[n]
            y = nx.get_node_attributes(graph,'y')[n]
            Sgroup.add_child(
                folium.Marker(
                    location=[y, x],
                    popup=n, icon=folium.Icon(color='blue')
                    )
                )
        Agroup.add_to(m)
        Sgroup.add_to(m)
        folium.LayerControl().add_to(m) 

        display(m)
        del m
        print('Alones: ',len(alone),alone) 
        print('Singles : ',len(single),single)  

    return alone, single





# Get Graph

In [ ]:
# file = os.path.join(CFG.data_path, f'[{CFG.area_name}]graph_with_known_tmja.graphml')
# try:
#     print(f'Load graph from disk for {CFG.area_name} roads,', end=' ')
#     G = ox.io.load_graphml(file)
#     print(f'{G.number_of_nodes()} nodes, {G.number_of_edges()} edges')

# except:
#     print('File not found')

In [ ]:
try:
    actual_level = joblib.load(os.path.join(CFG.data_path, 
                         f'[{CFG.area_name}]Gedges_last_interpolation_level.pkl'),
        )
    print('Continue on failure after level: ', actual_level)
except:
    actual_level = 0  
    print('First attempt')  

In [ ]:
edges = joblib.load( os.path.join(CFG.data_path, f'[{CFG.area_name}]Gedges_with_known_tmja_mthd2.pkl'))
display('Missing Tmja:',edges.shape[0],edges['tmja'].isna().value_counts(),'Tmja values:',edges['tmja'].value_counts())

In [ ]:
G = ox.graph_from_gdfs(
    joblib.load( os.path.join(CFG.data_path, f'[{CFG.area_name}]Gnodes_with_known_tmja_mthd1.pkl')), 
    edges
    )

del edges

# Interpolate missing tmja

In [ ]:
# edges = ox.graph_to_gdfs(G, nodes=False)
# display('Missing Tmja:',edges.shape[0],edges['tmja'].isna().value_counts(),'Tmja values:',edges['tmja'].value_counts())

In [ ]:
def missing_stats(graph):
    tmja_edges = dict()
    missing_edges = dict()
    for u, v, attr in graph.edges(data=True):
        state = attr.get('tmja', 'Missing')
        if state == 'Missing':
            missing_edges[f"({u}, {v})"] = state
        else:
            tmja_edges[f"({u}, {v})"] = state
    # print dict length
    print(f'Total edges: {graph.number_of_edges()}')
    # print tmja
    print(f'Tmja edges: {len(tmja_edges)}')
    # print missing
    print(f'Missing edges: {len(missing_edges)}')    

missing_stats(G)

## With number of neighbors in account

In [ ]:
for u, v, k, attr in G.edges(keys=True, data=True):
    print(u, v, k, attr)

    break

In [ ]:
# G1.nodes()[0]#['street_count']

In [ ]:
import networkx as nx
import numpy as np

def edges_states(graph, attribute='tmja'):
    missing_edges = dict()
    known_edges = dict()
    for u, v, k, attr in graph.edges(keys=True, data=True):
        state = attr.get(attribute, 0)
        if state == 0:
            missing_edges[(u, v, k)] = {
                attribute: state,
                'lenght': attr['length'],
                'u_roads': graph.nodes()[u]['street_count'],
                'v_roads': graph.nodes()[v]['street_count']  
            }
        else:
            known_edges[(u, v, k)] = {
                attribute: state,
                'lenght': attr['length'],
                'u_roads': graph.nodes()[u]['street_count'],
                'v_roads': graph.nodes()[v]['street_count']   
            }
    return missing_edges, known_edges      

In [ ]:
missing_edges, known_edges = edges_states(G)
print(len(known_edges), len(missing_edges))
# display(known_edges, missing_edges)

## Interpolation

In [ ]:
from statistics import mean

In [ ]:
n=0
for k,v in known_edges.items():
    print(k,v)
    n+=1
    if n ==10: break
n=0
for k,v in missing_edges.items():
    print(k,v)
    n+=1
    if n ==10: break

In [ ]:
G.edges[264590089, 264590099, 0]['tmja']

In [ ]:
pairs = []
for key, value in known_edges.items():
    pairs += [key, value]

pairs    

In [ ]:
for (a,b,c), v in sorted(known_edges.items(), key=lambda x: x[1]['tmja'], reverse=True):
    print(a,b,c, v['tmja'])

In [ ]:
import operator

def interpolation2(graph,  attribute='tmja', power=2.0, level=1):
    if level > 1 : print(f'Continue on failure at level {level - 1} ...')
    # Edges to find missing values
    to_interpolate, known_edges = edges_states(graph)
    last = 0
    progression = dict()
    # display(to_interpolate)
    # For each knownedge find neighboring edges with unknown attribute values
    added = 1
    while added > 0:
        added = 0
    # while (len(to_interpolate) - last ) != 0:
    #     last = len(to_interpolate)
        print(' '*100, end='\r')
        progression[level] = {
            'level': level,
            'knwon': len(known_edges),
            'unknown': len(to_interpolate)        
        }
        print(f'level: {level}, knwon: {len(known_edges)}, unknown: {len(to_interpolate)}')#, end='\r')
        inter_keys = list(to_interpolate.keys())

# in sorted(known_edges.items(), key=lambda x: x[1]['tmja'], reverse=True):
        sorted_items = sorted(known_edges.items(), key=lambda x: x[1]['tmja'], reverse=True)
        total = len(sorted_items)
        
        for n, ((u,v,k),value) in enumerate(sorted_items):
            print(f'{(n + 1) / total:.2%} ', end=' '*50 + '\r')
            # Heuristic to propagate the attribute value
            u_roads = int(graph.nodes()[u]['street_count'])
            v_roads = int(graph.nodes()[v]['street_count'])
            uv_tmja = float(graph.edges[u,v,k]['tmja'])
            # uv_lenght = float(graph.edges[u,v,k]['length'])
            tmja = uv_tmja / max(1, u_roads + v_roads)
            # print(u,v,k,value)
            
            candidates = nx.edges(graph, [u,v,k])
            # candidates = nx.bfs_edges(graph, source=u, reverse=False, depth_limit=2, sort_neighbors=None)
            # candidates = list(set(list(nx.dfs_edges(graph, source=u,  depth_limit=3))+list(nx.dfs_edges(graph, source=v, depth_limit=3))))
            # print('candidates:', candidates)

            

            for c in candidates:
                if (*c,0) in inter_keys:
                    if to_interpolate[(*c,0)][attribute] == 0:
                        # print('interpolation:', c)
                        edge_attrs = graph.edges[c[0],c[1],0]
                        # print(edge_attrs)
                        # edge_attrs.update({attribute: f'{tmja*(0.98**level)}'})
                        edge_attrs.update({attribute: tmja*(0.98**level)})

                        graph.add_edge(c[0],c[1],0, **edge_attrs)
                        # print(graph.edges[c[0],c[1],0])
                        added += 1
                if (*c,1) in inter_keys:        
                    if to_interpolate[(*c,1)][attribute] == 0:
                        # print('interpolation:', c)
                        edge_attrs = graph.edges[c[0],c[1],1]
                        # print(edge_attrs)
                        # edge_attrs.update({attribute: f'{tmja*(0.98**level)}'})
                        edge_attrs.update({attribute: tmja*(0.98**level)})

                        graph.add_edge(c[0],c[1],1, **edge_attrs) 
                        added += 1

        # Save new edges to cotinue interpolation on notebook failure 
        print(f'Save level {level} state...',end=' ') 
        joblib.dump(level,os.path.join(CFG.data_path, 
                         f'[{CFG.area_name}]Gedges_last_interpolation_level.pkl'),
        )
        joblib.dump(
            ox.utils_graph.graph_to_gdfs(graph, nodes=False, edges=True, node_geometry=True, fill_edge_geometry=True), 
            os.path.join(CFG.data_path, 
                         f'[{CFG.area_name}]Gedges_with_known_tmja_mthd2.pkl'),
                         compress=3)
                      
        print('done.')

        to_interpolate, known_edges = edges_states(graph)
        level +=1
        # print(f'Added: {added}                              ')
        
    return graph , progression       


Gi, progression = interpolation2(G.copy(), level=actual_level + 1)


In [ ]:
# # Debug
# from shapely import box
# marne = box(3.29590,49.50024,5.13611,48.42191)
# G2 = ox.truncate.truncate_graph_bbox(
#     G,
#     49.50024,48.42191,5.13611,3.29590,
#     retain_all=False
# )

# print('interpolate on Marne')
# A, B = interpolation2(G2.copy())
# # end of debug



In [ ]:
pd.DataFrame.from_dict(progression, orient='index').plot(kind='line')

In [ ]:
if CFG.viz:
    graphplot(Gi, edg_attr='tmja')

In [ ]:
if CFG.viz:
    m = ox.folium.plot_graph_folium(Gi, tiles='OpenStreetMap',
                                    popup_attribute ='tmja',)
    display(m)
    del m                                

In [ ]:
issues, _ = edges_states(Gi)
issues

## Save graph

In [ ]:
# Define the file namer the graph file
# area_name = [s.replace("\'","@") for s in selection]
filename = os.path.join(CFG.data_path, f'[{CFG.area_name}]graph_tmja interpolated.graphml"))

ox.io.save_graphml(G, filename)


_____________________________

In [ ]:
******* Fin du taritement des données **************

In [ ]:
node_gdf, edges_gdf = map_load_graph_gdfs(filename)

In [ ]:
node_gdf.sample(10).T

In [ ]:
edges_gdf.sample(10).T